<a href="https://colab.research.google.com/github/groupe11ML/groupe11ML/blob/main/d%C3%A9tection_des_spams_TP3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GROUPE 11 
#KEUFACK DONGMO BELVIANE BERENICE 19Y204
#LEUKAM FERMAT 19Y208
#LONTSI TIDOH MOREL 19Y082
#TCHAMEDEU TCHAMEDEU JORDAN STEVE 19Y458

Dans le cadre de la classification des emails (spam ou ham), les caractéristiques à comparer sont les fréquences d'un mot dans chaque email. La distance euclidienne est utilisée pour déterminer la similarité entre deux e-mails ; plus la distance est petite, plus elle est similaire. 

PROCEDURE :
1. Chargez les e-mails de spam et de jambon
2. Supprimer la ponctuation et les symboles courants
3. Minuscule toutes les lettres
4. Supprimez les mots vides (mots très courants comme les pronoms, les articles, etc.)
5. Divisez les e-mails en e-mails de formation et en e-mails de test
6. Pour chaque e-mail de test, calculez la similarité entre celui-ci et tous les e-mails de formation
    6.1. Pour chaque mot qui existe dans l'e-mail de test ou l'e-mail de formation, comptez sa fréquence dans les deux e-mails
    6.2. calculer la distance euclidienne entre les deux e-mails pour déterminer la similarité
7. Trier les e-mails par ordre croissant de distance euclidienne
8. Sélectionnez les k voisins les plus proches (distance la plus courte)
9. Attribuez la classe la plus fréquente dans les k plus proches voisins sélectionnés au nouvel e-mail

Bibliothèques utilisées :

In [ ]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

bibliothèque os pour ouvrir et lire des fichiers.
bibliothèque string pour la liste de ponctuation
stopwords contient la liste des mots vides.
train_test_split pour diviser les données en données d'apprentissage et de test.
accuracy_score pour calculer la précision des algorithmes.
numpy pour permettre une manipulation avancée des tableaux.

Loading the Data

In [ ]:
def load_data():
    print("Loading data...")
    
    ham_files_location = os.listdir("dataset/ham")
    spam_files_location = os.listdir("dataset/spam")
    data = []
    # Load ham email
    for file_path in ham_files_location:
    f = open("dataset/ham/" + file_path, "r")
    text = str(f.read())
    data.append([text, "ham"])
      
  # Load spam email
    for file_path in spam_files_location:
    f = open("dataset/spam/" + file_path, "r")
    text = str(f.read())
    data.append([text, "spam"])
    data = np.array(data) 
    
    print("drapeau 1 : données chargées") 
    return data

os.listdir() renvoie une liste de tous les noms de fichiers à l'intérieur d'un dossier. Ceci est utilisé pour récupérer tous les noms de fichiers des fichiers texte dans chacun des dossiers ham et spam et les stocker respectivement dans ham_files_location et spam_files_location. data est une liste pour stocker chaque texte d'e-mail et son étiquette correspondante.


Les données de la liste sont transformées en un tableau numpy, pour permettre une meilleure manipulation du tableau ultérieurement. les données sont ensuite renvoyées.

#Prétraitement des données

punc contient une liste de ponctuation et de symboles
sw contient une liste de mots vides de la bibliothèque nltk.corpus

Pour chaque enregistrement dans data, pour chaque élément (symbole ou ponctuation) dans punct, remplacez l'élément par une chaîne vide, pour supprimer l'élément de record[0] (chaîne de texte d'e-mail).

In [ ]:
# Preprocessing data: noise removal
def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           # Punctuation list
    sw = stopwords.words('english')     # Stopwords list
    for record in data:
        # Remove common punctuation and symbols
        for item in punc:
            record[0] = record[0].replace(item, "")
        # Lowercase all letters and remove stopwords 
        splittedWords = record[0].split()
        newText = ""
        for word in splittedWords:
            if word not in sw:
                word = word.lower()
                newText = newText + " " + word  
        record[0] = newText
        
    print("flag 2: preprocessed data")        
    return data

Utilisez la méthode split () sur l'enregistrement de texte de l'e-mail [0] pour renvoyer une liste de tous les mots de l'e-mail. Parcourez cette liste de mots, et si le mot n'est pas dans la liste des mots vides, mettez-le en minuscules et ajoutez le mot à newText. newText contiendra l'e-mail mais vide de mots vides. newText est réattribué à record[0]. Une fois que chaque enregistrement[0] a été prétraité, les données nettoyées sont renvoyées.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Fractionnement des données en ensembles d'apprentissage et de test
L'ensemble de données est divisé en un ensemble d'apprentissage (73 %) et un ensemble de test (27 %).

In [ ]:
# Splitting original dataset into training dataset and test dataset
def split_data(data):
    print("Splitting data...")
    
    features = data[:, 0]   # array containing all email text bodies
    labels = data[:, 1]     # array containing corresponding labels
    print(labels)
    training_data, test_data, training_labels, test_labels =train_test_split(features, labels, test_size = 0.27, random_state = 42)
    
    print("flag 3: splitted data")
    return training_data, test_data, training_labels, test_labels

Tout d'abord, il était nécessaire d'avoir les textes des e-mails dans un tableau qui leur était propre et les étiquettes dans un autre tableau qui leur était propre. Ainsi, les textes des e-mails étaient stockés dans des fonctionnalités et les étiquettes étaient stockées dans des étiquettes. La méthode train_test_split a ensuite été utilisée pour diviser les données en training_data, test_data, training_labels et test_labels. L'état aléatoire a été défini sur 42 pour garantir que la même sortie de mélange aléatoire sera obtenue à des fins de test. Après le fractionnement, training_data, test_data, training_labels et test_labels sont renvoyés.

#The KNN Algorithm

fonction get_count()

In [ ]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

Cette fonction prend un seul texte d'e-mail et le divise à l'aide de split(). La fréquence d'occurrence de chaque mot dans l'e-mail est comptée et enregistrée dans wordCounts, qui est de type dictionnaire. Le dictionnaire wordCounts est ensuite renvoyé.

euclidean_difference() function

sa fonction prend en compte un dictionnaire de nombre de mots d'un e-mail de test test_WordCounts, et un autre dictionnaire d'un e-mail de formation, training_wordCounts. total stocke la somme de la différence au carré de la fréquence d'un mot dans l'e-mail de test et d'entraînement.

In [ ]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2
            del training_WordCounts[word]
        else:
            total += test_WordCounts[word]**2
    for word in training_WordCounts:
            total += training_WordCounts[word]**2
    return total**0.5

Tout d'abord, nous parcourons les mots du dictionnaire d'e-mails de test. Pour chaque mot, il y a trois cas. Le premier cas est qu'il existe à la fois dans l'e-mail de test et dans l'e-mail de formation. Dans ce cas, le total est incrémenté de la différence au carré de la fréquence du mot dans l'e-mail de test et l'e-mail de formation.

Ensuite, le mot commun est supprimé du dictionnaire des e-mails de formation, pour accélérer la prochaine boucle for

Le deuxième cas est que le mot est uniquement dans l'e-mail de test. Dans ce cas, il n'est pas nécessaire de trouver la différence (puisque sa fréquence est de 0 dans l'e-mail de formation), nous ajoutons donc simplement la fréquence au carré du mot au total.

Le dernier cas est que le mot est uniquement dans l'e-mail de formation. Étant donné que nous avons supprimé tous les mots courants dans la boucle for précédente, nous parcourons simplement le dictionnaire des e-mails de formation et ajoutons la fréquence au carré de chaque mot au total.

Enfin, la racine carrée du total (la racine carrée de la somme de la différence au carré des fréquences de chaque mot) est renvoyée sous la forme d'un double. C'est la fin de la fonction de calcul de la distance euclidienne.

#get_class() function

Cette fonction prend en compte la liste des K plus proches voisins sélectionnés pour déterminer la classe de l'email de test en cours. spam_count et ham_count stockent la fréquence d'occurrence de chaque étiquette "spam" et étiquette "ham" respectivement dans les K voisins les plus proches sélectionnés.

In [ ]:
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
        if spam_count > ham_count:
          return "spam"
        else:
          return "ham"

En utilisant une boucle for, pour chaque valeur parmi les K valeurs sélectionnées, si la valeur de l'étiquette [0] est égale à "spam", alors spam_count est incrémenté de 1. Sinon, ham_count est incrémenté de 1

Après la boucle, si spam_count est supérieur à ham_count, cela signifie que l'e-mail de test actuel a plus tendance à être du spam, donc une chaîne "spam" est renvoyée comme étiquette prédite. Sinon, la chaîne "ham" est renvoyée comme étiquette prédite.

#knn_classifier() function

C'est la fonction de classificateur KNN. Il prend en compte l'e-mail de formation, les étiquettes de formation, les données de test, la valeur K et le nombre d'e-mails de test à tester sur les 27 % d'e-mails de test d'origine. result est la liste qui contiendrait les étiquettes prédites. compteur sera simplement utilisé à des fins d'affichage pour indiquer la progression lorsque le programme est exécuté.

In [ ]:
def knn_classifier(training_data, training_labels, test_data, K, tsize):
    print("Running KNN Classifier...")
    
    result = []
    counter = 1
    # word counts for training email
    training_WordCounts = [] 
    for training_text in training_data:
            training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        similarity = [] # List of euclidean distances
        test_WordCounts = get_count(test_text)  # word counts for test email
        # Getting euclidean difference 
        for index in range(len(training_data)):
            euclidean_diff =euclidean_difference(test_WordCounts, training_WordCounts[index])
            similarity.append([training_labels[index], euclidean_diff])
        # Sort list in ascending order based on euclidean difference
        similarity = sorted(similarity, key = lambda i:i[1])
        # Select K nearest neighbours
        selected_Kvalues = [] 
        for i in range(K):
            selected_Kvalues.append(similarity[i])
        # Predicting the class of email
        result.append(get_class(selected_Kvalues))
        return result

Étant donné que l'ensemble de formation est constant, nous pouvons compter une fois pour toutes la fréquence des mots dans chaque e-mail de formation. Ainsi, pour chaque texte d'e-mail dans les données d'apprentissage, son dictionnaire de fréquence de mots est obtenu à l'aide de get_count(). Le dictionnaire est ensuite ajouté à la liste training_WordCounts pour être stocké.


Désormais, pour chaque e-mail de test dans les données de test, les opérations suivantes sont effectuées. Une similarité de liste vide est déclarée. Il stockera les distances euclidiennes entre l'e-mail de test actuel et chaque e-mail d'entraînement. Ensuite, le dictionnaire de fréquence des mots de l'e-mail de test est obtenu à l'aide de get_count().

Puisque nous avons déjà les dictionnaires de fréquence des mots de tous les e-mails de formation et l'e-mail de test actuel. Nous pouvons continuer à calculer la distance euclidienne entre l'e-mail de test actuel et chaque e-mail d'entraînement à l'aide d'une boucle for qui itère x fois, où x est égal à la taille de l'ensemble de données d'entraînement. Après chaque itération, la distance euclidienne calculée est ajoutée à la liste de similarité, ainsi que l'étiquette correspondante de l'e-mail d'entraînement.

Après que toutes les distances euclidiennes ont été stockées. Nous trions la liste de similarité par ordre croissant en fonction de la deuxième colonne, c'est-à-dire en fonction de la distance euclidienne (du plus proche au plus éloigné).

Maintenant, puisque la liste de similarité est déjà triée, nous pouvons facilement ajouter les K voisins les plus proches à la liste selected_Kvalues ​​en utilisant une simple boucle for.

Enfin, et avant de passer au prochain e-mail de test. Nous déterminons la classe de l'e-mail de test actuel à l'aide de get_class(). Maintenant, nous avons atteint la fin d'une itération, et la prochaine itération peut commencer à classer le prochain e-mail de test.

Une fois que tous les e-mails de test ont été classés et que la boucle for a atteint sa fin, la liste des résultats, qui contient la liste des étiquettes prédites, est renvoyée.

#main() function

In [ ]:
def main(K):
    data = load_data()
    data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize) 
    accuracy = accuracy_score(test_labels[:tsize], result)
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t\t: " + str(len(test_data)))
    print("K value\t\t\t\t: " + str(K))
    print("Samples tested\t\t: " + str(tsize))
    print("% accuracy\t\t\t: " + str(accuracy * 100))
    print("Number correct\t\t: " + str(int(accuracy * tsize)))
    print("Number wrong\t\t: " + str(int((1 - accuracy) * tsize)))
main(11)

size spécifie le nombre d'e-mails de test (sur les 27 % de données de test d'origine) pour prédire leurs étiquettes. Actuellement, tsize est défini pour être égal à l'ensemble des e-mails de test.

Maintenant, la fonction knn_classifier() est appelée pour prédire les étiquettes des e-mails de test. La liste renvoyée des étiquettes prédites est stockée dans le résultat. Après cela, la précision est calculée à l'aide de la méthode precision_score() de la bibliothèque sklearn. Cette méthode compare la liste d'étiquettes réelle test_labels avec le résultat de la liste d'étiquettes prédite.

Ces lignes affichent les détails de l'exécution tels que la taille des données de formation, la taille des données de test, la valeur K, le nombre d'échantillons testés, le pourcentage de précision, le nombre d'e-mails correctement identifiés et le nombre d'e-mails faussement identifiés.

finalement, c'est la ligne qui initie le programme en appelant la fonction main, et lui donne la valeur K (qui est 11 dans ce cas).